# Class Anomaly Detection QAOA (AD_QAOA)

This class can solve the Anomaly Detection problem for time series analysis formulated as a QUBO problem and visualize the results.

**class AD_QAOA** `(self,
             X,
             alpha = -0.5,
             beta = 0.5,
             radius = 1,
             top_n_samples = 5,
             num_iterations = 10,
             model_name = 'quadratic',
             model_params = {},
             radius_adjustment = False,
             num_layers = 1,
             debug=False)`

#### Parameters

* `X` (List[Tuple[int, float]]): Time series data.
* `alpha` (float): Weight for the linear terms in the QUBO problem.
* `beta` (float): Weight for the quadratic terms in the QUBO problem.
* `radius` (float): Radius for the covering boxes.
* `top_n_samples` (int): Number of top samples to consider.
* `num_iterations` (int): Number of iterations for the COBYLA optimizer.
* `model_name` (str): Model selected for the detection pipeline.
* `model_params` (str): Model's parameters (if any).
* `radius_adjustment` (bool):  Enables the radius adjustment mechanism for the set covering.
* `num_layers` (int): Number of layers (p) to use in QAOA.
* `debug` (bool): Enables some debug prints throught the code.

## Requirements

`pip install qiskit`

`pip install qiskit. aer`

`pip.install qiskit-optimization`

`pip install qiskit-optimization[cplex]`

## Imports

In [8]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, NELDER_MEAD, ADAM
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer, MinimumEigenOptimizer
import numpy as np
import math
from typing import Tuple, List, Dict
from matplotlib import pyplot as plt
from statistics import mean
import random
from sklearn.linear_model import RANSACRegressor
from sklearn.pipeline import make_pipeline
import math
from scipy.optimize import curve_fit
import itertools

## Attributes

### initialization

Initializes the AD_QAOA class.  
Disclaimer: mixer selection work in progress.

`__init__(self,
             X,
             alpha,
             beta,
             radius,
             top_n_samples,
             num_iterations,
             model_name,
             model_params,
             radius_adjustment,
             num_layers,
             debug)`

#### Args

* `self` : instanciating the attribute.
* `X` (List[Tuple[int, float]]): Time series data.
* `alpha` (float): Weight for the linear terms in the QUBO problem.
* `beta` (float): Weight for the quadratic terms in the QUBO problem.
* `radius` (float): Radius for the covering boxes.
* `top_n_samples` (int): Number of top samples to consider.
* `num_iterations` (int): Number of iterations for the COBYLA optimizer.
* `model_name` (str): Model selected for the detection pipeline.
* `model_params` (str): Model's parameters (if any).
* `radius_adjustment` (bool):  Enables the radius adjustment mechanism for the set covering.
* `num_layers` (int): Number of layers (p) to use in QAOA.
* `debug` (bool): Enables some debug prints throught the code.

### matrix M

Builds the matrix M for the QUBO Anomaly Detection objective function. 

`matrix_M(self)`

#### Args

* `self`

#### Returns

* `M` (np.ndarray): the matrix M of the Anomaly Detection QUBO problem, combination of both linear and quadratic terms. 

### off diag M

Builds the off-diagonal terms (Q, quadratic contribution for the QUBO problem) for the corresponding M matrix.

`off_diag_M (self, data)`

####  Args

* `self`
* `data` (List[Tuple[int, float]]): Time Series data.


#### Returns

* `Q` (np.ndarray): The symmetric matrix Q of the quadratic terms for the QUBO problem.


### compute delta

Creates the diagonal terms (delta, linear contribution for the QUBO problem) for the corresponding M matrix, computing the difference between the data sample and the corresponding model fitting values.

`compute_delta (self, data, model_values)`

#### Args

* `self`
* `data` (np.ndarray): Time Series data.
* `model_values` (np.ndarray): Corresponding model values for the fitting.

#### Returns

* `List[float]`: List of absolute differences data sample/model sample.

### inverse transform

Applies an inverse transformation to delta values. Larger values ​​of delta become smaller, and vice versa. This allows the minimization problem for the QUBO formulation to correctly identify anomalies based on the highest values in the model fitting vector (high differences bewteen the model and the data sample). Also works as a normalization for the values.

`inverse_transform(self, delta_values, scale_factor = 0.5)`

#### Args

* `self`
* `delta_values` (List[float]): List of absolute differences data sample/model sample.
* `scale_factor` (float): Scaling factor for the normalization and the transformation of the values.

#### Returns

* `transfrmed_values` (List[float]): List of transformed values to be used in the building of the diagonal of M for the QUBO problem.

### diag M

Computes the diagonal of the matrix M, using the normalization and the inversion of the data values, having selected a proper fitting model.  
Adjustable scale factor: for a standard range of values use default; for very small dataset values use smaller factors.

`diag_M(self, data, scale_factor = 0.5)`

#### Args

* `self`
* `data` (np.ndarray): Time Series data.
* `scale_factor` (float): Scaling factor for the normalization and the transformation of the values.

#### Returns

* `transformed_delta` (List(float)): The diagonal component of the effective matrix M for the QUBO problem.

#### Raises

* `ValueError`: If a not supported model type is selected.

### distance

Calculates the selected distance between two points (default is absolute distance).

`distance(self, point1, point2, kind = "absolute_difference")`

#### Args

* `self`
* `point1` (np.ndarray): First point.
* `point2` (np.ndarray): Second point.
* `kind` (str): the kind of distance to be used. Default is "absolute".

#### Returns

* `float`: the distance between point1 and point2.

#### Raises

* `NotImplementedError`: If a not supported distance type is selected.
* `ValueError`: if at least one of the points is not in the bidimensional array form (time_series' data point).

### moving average expanded

Calculates the moving average model (expanded to match the lenght of the list of samples) for the group of sample given a selected window size (default is 2).

`moving_average_expanded(self, series, window_size = 2)`

#### Args

* `self`
* `series` (np.ndarray): Data samples used.
* `window_size` (int): The shifting window size for the computation.

#### Returns

* `expanded_moving_avg` (np.ndarray): Vector of the moving average model (with position currespondance on the original data samples).

### plot model

Creates the plotting showcasing the model fitting on the Time Series.

`plot_model(self)`

#### Args

* `self`

#### Raises

* `ValueError`: If a non supported model is selected.

### plot time series

Creates the plotting showcasing the Time Series.

`plot_time_series(self)`

#### Args

* `self`

### plot distance

Creates the plotting showcasing the distance between a selected sample (first_sample_value) and the rest of the Time Series (batch).

`plot_distances_with_arrows(self)`

#### Args

* `self`

### radius adj

Radius adjustment algorithm. Determinates the best radius value for the centers and batch in exam. Default the radius is set to 1.00 and then is tried enlarging or reducing for the covering achievement.  
Adjust the exclusive tolerance (0.33 standard) for the max_non_center_value.  
Adjust the inclusive tolerance ( /2 standard) for the normal values inclusion.

#### Args

* `self`
* `centers`: List of centers selected as 1s in the QAOA quantum state solution.

#### Returns

* `self.radius` (float): The optimal radius value identified for the batch at hand.


### solve qubo

Solves the QUBO Anomaly Detection problem using the parameters defined in the class.

`solve_qubo(self)`

#### Args

* `self`

#### Returns

* `top_n_states` (List[List[int]]): The best binary solutions found for the QUBO problems.
* `variables_dict` (dict): A dictionary with the values of the variables.

### centers storage

Stores the center coordinates corresponding to 1s in the QAOA solution (default is the first state if none is provided).

`centers_storage(self, state=None)`

#### Args

* `self`
* `state` (List[int], optional): A binary list representing the QAOA solution state. If None, the function will use the first quantum state (most counts).

#### Returns

* `centers` (List[Tuple[int, float]]): A list of tuples, where each tuple contains a timestamp and its corresponding value for each selected center. These centers are identified by the positions in the solution string where the value is '1'.

### detect anomalies

Detects anomalies based on the selected centers in the QAOA solution state.

`detect_anomalies(self, state=None)`

#### Args

* `self`
* `state` (List[int], optional): A binary list representing the QAOA solution state. If None, the function will use the first quantum state (most counts).

#### Returns

* `boxes` (List[Tuple[int, float]]): A list of tuples representing the coverage spheres for each center, where each box is centered around selected points and adjusted to cover surrounding data points.

### associate centers with radius

Associates each center with the corresponding calculated radius.

`associate_centers_with_radius(self, state=None)`

#### Args

* `self` 
* `state` (list, optional): A binary list representing the QAOA solution state. If None, the function will retrieve the best solution available.

#### Returns

* `centers_with_radius` (list of tuples): A list of (center, radius) pairs, where each center has an associated radius.

### visualize anomalies

Visualizes anomalies detected by the QAOA model, highlighting centers and coverage areas on a scatter plot.  
Adjust the plot title at will. 

`visualize_anomalies(self, state=None)`

#### Args

* `self`
* `state` (List[int], optional): A binary list representing the QAOA solution state. If None, the function will use the first quantum state (most counts).

### cost function

Calculates the cost for a given QAOA solution state based on the matrix M.

`cost_function(self, M, state)`

#### Args

* `self`
* `M`(np.ndarray): A square symmetric matrix representing interactions between variables. The diagonal elements represents linear terms, while the off-diagonal elements represent quadratic terms.
* `state` (np.ndarray): A binary vector (1D array) representing the QAOA solution state. The length of "state" should match the dimensions of "M".

#### Returns

* `cost` (float): The calculated cost for the given state, based on the weighted sum of linear and quadratic terms.

#### Raises

* `ValueError`: If the dimension of `state` does not match the dimension of "M".

### find max cost

Finds the state with the maximum cost for a given matrix M.

`find_max_cost(self, M)`

#### Args

* `self`
* `M` (np.ndarray): A square symmetric matrix representing interactions between variables, with diagonal elements representing linear terms and off-diagonal elements representing quadratic terms.

#### Returns

* `best_state` (np.ndarray): A binary array representing the state that maximizes the cost function.
* `max_cost` (float): The maximum cost value obtained with the best state.

### find min cost

Finds the state with the minimum cost for a given matrix M.

`find_min_cost(self, M)`

#### Args

* `self`
*  `M` (np.ndarray): A square symmetric matrix representing interactions between variables, with diagonal elements representing linear terms and off diagonal elements representing quadratic terms.

#### Returns

* `best_state` (np.ndarray): A binary array representing the state that minimizes the cost function.
*  `min_cost` (float): The minimum cost value obtained with the best state.